Data can be sourced from [Brisbane City Council Library Checkouts](https://www.data.brisbane.qld.gov.au/data/dataset/library-checkouts-branch-date)

1. Set up the session
2. Set up the context
3. Raise the log level

#### Setup the SparkSession and SparkContext

In [1]:
from pyspark.sql import SparkSession, functions as F, types as T, Window

spark = (SparkSession
            .builder
            .appName("Demo")
            .getOrCreate())

sc = spark.sparkContext
sc.setLogLevel("ERROR")

22/10/06 17:34:57 WARN Utils: Your hostname, spyro-IdeaPad-5-15ITL05 resolves to a loopback address: 127.0.1.1; using 192.168.43.211 instead (on interface wlp0s20f3)
22/10/06 17:34:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/06 17:34:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### Import the data

1. Read in the csv
2. Print the record count
3. Print the schema
4. Print 3 records

In [2]:
df_in = spark.read.csv("library_checkouts_202206.csv", header=True, quote="\"", escape="\"").drop("status", "language")

print(f"Number of records: {df_in.count()}\n")

df_in.printSchema()

df_in.show(3)

Number of records: 59593

root
 |-- title: string (nullable = true)
 |-- author: string (nullable = true)
 |-- call_number: string (nullable = true)
 |-- item_id: string (nullable = true)
 |-- item_type_code: string (nullable = true)
 |-- age: string (nullable = true)
 |-- checkout_library: string (nullable = true)
 |-- date: string (nullable = true)

+--------------------+-------------+-----------+--------------+--------------+--------+----------------+--------------+
|               title|       author|call_number|       item_id|item_type_code|     age|checkout_library|          date|
+--------------------+-------------+-----------+--------------+--------------+--------+----------------+--------------+
|"Are we nearly th...|         null|821.008 ARE|34000061508842|    NONFICTION|JUVENILE|             TWG|20220607131747|
| "D" is for deadbeat|Grafton, Sue,|    CD-BOOK|34000099825432|       CD-BOOK|   ADULT|             ASH|20220607105013|
| "F" is for fugitive|Grafton, Sue,|    CD-BOO

#### Data pre-processing

1. Read in a mapping for library_code -> library_name
2. Cast the date column to timestamp
3. Apply the library_code mapping
4. Check the date column was correctly remapped
4. Print 3 records

In [3]:
library_map = spark.read.csv("library_mapping.csv", header=True)

df = (df_in.withColumn("date", F.to_timestamp(F.col("date"), "yyyyMMddHHmmss"))
        .withColumnRenamed("checkout_library", "library_code")
        .join(library_map, "library_code", "left"))

print(df.schema["date"], "\n")

df.show(3)

StructField('date', TimestampType(), True) 

+------------+--------------------+-------------+-----------+--------------+--------------+--------+-------------------+------------+
|library_code|               title|       author|call_number|       item_id|item_type_code|     age|               date|library_name|
+------------+--------------------+-------------+-----------+--------------+--------------+--------+-------------------+------------+
|         TWG|"Are we nearly th...|         null|821.008 ARE|34000061508842|    NONFICTION|JUVENILE|2022-06-07 13:17:47|     Toowong|
|         ASH| "D" is for deadbeat|Grafton, Sue,|    CD-BOOK|34000099825432|       CD-BOOK|   ADULT|2022-06-07 10:50:13|    Ashgrove|
|         ASH| "F" is for fugitive|Grafton, Sue,|    CD-BOOK|34000099825382|       CD-BOOK|   ADULT|2022-06-07 10:50:14|    Ashgrove|
+------------+--------------------+-------------+-----------+--------------+--------------+--------+-------------------+------------+
only showing top 

#### Explore the libraries

1. Print the number of libraries
2. Print the top 3 libraries by number of checkouts

In [4]:
library_count = (df.select("library_name")
                   .distinct()
                   .count())

print(f"Distinct libraries: {library_count}\n")

print("Top 3 libraries by number of checkouts")

(df.groupBy("library_name")
   .count()
   .orderBy("count", ascending=False)
   .show(3))

Distinct libraries: 36

Top 3 libraries by number of checkouts
+---------------+-----+
|   library_name|count|
+---------------+-----+
|      Chermside| 4292|
|Sunnybank Hills| 3747|
|       Ashgrove| 3592|
+---------------+-----+
only showing top 3 rows



#### Which titles are the most popular?

1. Print the top 10 titles by number of checkouts

In [5]:
(df.groupBy("title", "item_type_code")
   .count()
   .orderBy("count", ascending=False)
   .show(10, truncate=False))

+------------------------------+--------------+-----+
|title                         |item_type_code|count|
+------------------------------+--------------+-----+
|The New idea.                 |AD-MAGS       |135  |
|Woman's day.                  |AD-MAGS       |133  |
|The Australian women's weekly.|AD-MAGS       |118  |
|Who weekly.                   |AD-MAGS       |116  |
|Hello                         |AD-MAGS       |114  |
|New scientist. (2021 onwards) |AD-MAGS       |111  |
|Australian house and garden.  |AD-MAGS       |105  |
|FASTBACK - The bad guys       |FBKIDS        |98   |
|Country style                 |AD-MAGS       |84   |
|Australian home beautiful.    |AD-MAGS       |83   |
+------------------------------+--------------+-----+
only showing top 10 rows



#### What is the most popular title in each item type?

1. Get the checkout count by title and category
2. Rank the titles within each category type
3. Select the top ranked title (in each category)
4. Print the top 10 top ranked titles

In [6]:
(df.groupBy("title", "item_type_code")
   .count()
   .withColumn("rank", F.rank().over(Window.partitionBy("item_type_code").orderBy(F.desc("count"))))
   .filter(F.col("rank") == 1)
   .drop("rank")
   .orderBy("count", ascending=False)
   .show(10, truncate=False))

+---------------------------------------------------------+--------------+-----+
|title                                                    |item_type_code|count|
+---------------------------------------------------------+--------------+-----+
|The New idea.                                            |AD-MAGS       |135  |
|FASTBACK - The bad guys                                  |FBKIDS        |98   |
|The bad guys.                                            |JU-PBK        |71   |
|Bluey.                                                   |DVD           |49   |
|One piece.                                               |GRAPHICNOV    |39   |
|Pok�mon Magazine                                         |JU-MAGS       |36   |
|Sparring partners /                                      |AD-PBK        |31   |
|Independent and unofficial guide to your minecraft world.|YA-MAGS       |29   |
|Ming pao weekly                                          |LOTE-MAG      |21   |
|FASTBACK - Sparring partner

#### What does the borrowing pattern look like for a specific library?

1. Print 20 consecutive checkouts

In [7]:
df_exp = df.select("library_code", "library_name", "title", "author", "item_type_code", "age", "date")

(df_exp.orderBy("library_code", "date").drop("title")
    .show(truncate=False))

+------------+------------+------------------+--------------+--------+-------------------+
|library_code|library_name|author            |item_type_code|age     |date               |
+------------+------------+------------------+--------------+--------+-------------------+
|ANN         |Annerley    |Fontana, Shea,    |JU-PBK        |JUVENILE|2022-06-07 06:22:16|
|ANN         |Annerley    |Doyle, Glennon,   |BIOGRAPHY     |ADULT   |2022-06-07 06:22:23|
|ANN         |Annerley    |Marchetta, Melina,|JU-PBK        |JUVENILE|2022-06-07 06:23:36|
|ANN         |Annerley    |Loreau, Dominique,|NONFICTION    |ADULT   |2022-06-07 08:46:36|
|ANN         |Annerley    |null              |NONFICTION    |ADULT   |2022-06-07 08:46:36|
|ANN         |Annerley    |null              |AD-MAGS       |ADULT   |2022-06-07 08:46:36|
|ANN         |Annerley    |Damas, Jeanne,    |NONFICTION    |ADULT   |2022-06-07 08:46:36|
|ANN         |Annerley    |Obama, Michelle,  |BIOGRAPHY     |ADULT   |2022-06-07 09:04:20|

#### Can we group sets of checkouts together?

In [8]:
def group_records_into_checkouts(df, allowed_gap_in_seconds, show_intermediate=True):
    # Add a row number to act as a tie break when ordering identical dates
    df_checkout = df.withColumn("row_number", F.row_number().over(Window.partitionBy("library_code").orderBy("date")))

    # Grab the time of the immediately previous checkout
    df_checkout = df_checkout.withColumn("previous_checkout", F.lag(F.col("date")).over(Window.partitionBy("library_code").orderBy("date", "row_number")))

    # Only consider it to be the same checkout if the previous occurred within the previous X seconds
    df_checkout = (df_checkout.withColumn("time_between_checkouts", F.unix_timestamp("date") - F.unix_timestamp("previous_checkout"))
                    .withColumn("is_same_checkout", F.col("time_between_checkouts") <= allowed_gap_in_seconds)
                    .withColumn("new_checkout_increment", F.when(F.col("is_same_checkout"), 0).otherwise(1))
                    .drop("is_same_checkout"))

    # The checkout IDs can be generated by doing a cumulative sum 
    df_checkout = df_checkout.withColumn("checkout_id", F.sum("new_checkout_increment")
                                            .over(Window.partitionBy("library_code").orderBy("date", "row_number").rangeBetween(Window.unboundedPreceding, 0)))

    if show_intermediate:
        df_checkout.drop("title", "author", "item_type_code", "age").orderBy("library_code", "date", "row_number").show(5)

    return df_checkout.drop("row_number", "previous_checkout", "new_checkout_increment")

1. Group the records into clusters with unique checkout_ids
2. Print statistics about the checkout_ids

In [9]:
df_checkout = group_records_into_checkouts(df_exp, 5)

(df_checkout.groupBy("library_code", "checkout_id")
                       .count()
                       .select("count")
                       .describe()
                       .show())

+------------+------------+-------------------+----------+-------------------+----------------------+----------------------+-----------+
|library_code|library_name|               date|row_number|  previous_checkout|time_between_checkouts|new_checkout_increment|checkout_id|
+------------+------------+-------------------+----------+-------------------+----------------------+----------------------+-----------+
|         ANN|    Annerley|2022-06-07 06:22:16|         1|               null|                  null|                     1|          1|
|         ANN|    Annerley|2022-06-07 06:22:23|         2|2022-06-07 06:22:16|                     7|                     1|          2|
|         ANN|    Annerley|2022-06-07 06:23:36|         3|2022-06-07 06:22:23|                    73|                     1|          3|
|         ANN|    Annerley|2022-06-07 08:46:36|         4|2022-06-07 06:23:36|                  8580|                     1|          4|
|         ANN|    Annerley|2022-06-07 08:

1. Calculate some checkout cluster statistics (count, duration)
2. Get the checkout cluster with the largest number of checkouts
3. Print the cluster stats
3. Print all of the records in that cluster

In [10]:
top_record = (df_checkout.groupBy("library_code", "checkout_id")
                         .agg(F.count("library_code").alias("count"), 
                              F.max("date").alias("max_date"),
                              F.min("date").alias("min_date"))
                         .withColumn("duration", F.unix_timestamp(F.col("max_date")) - F.unix_timestamp(F.col("min_date")))
                         .orderBy("count", ascending=False)
                         .limit(1))

top_record.show()

biggest_checkout = df_checkout.join(top_record, ["library_code", "checkout_id"])

(biggest_checkout.drop("library_code", "library_name", "count", "min_date", "max_date", "duration")
                 .orderBy("date")
                 .show(100, truncate=False))


+------------+-----------+-----+-------------------+-------------------+--------+
|library_code|checkout_id|count|           max_date|           min_date|duration|
+------------+-----------+-----+-------------------+-------------------+--------+
|         SBK|        293|   62|2022-06-07 15:28:15|2022-06-07 15:27:26|      49|
+------------+-----------+-----+-------------------+-------------------+--------+

+-----------+----------------------------------------------------------------------------+----------------------+--------------+--------+-------------------+----------------------+
|checkout_id|title                                                                       |author                |item_type_code|age     |date               |time_between_checkouts|
+-----------+----------------------------------------------------------------------------+----------------------+--------------+--------+-------------------+----------------------+
|293        |Money magazine.                  

In [11]:
df.explain(True)

== Parsed Logical Plan ==
'Join UsingJoin(LeftOuter,Buffer(library_code))
:- Project [title#17, author#18, call_number#19, item_id#20, item_type_code#21, age#24, checkout_library#25 AS library_code#130, date#121]
:  +- Project [title#17, author#18, call_number#19, item_id#20, item_type_code#21, age#24, checkout_library#25, to_timestamp(date#26, Some(yyyyMMddHHmmss), TimestampType, Some(Australia/Sydney)) AS date#121]
:     +- Project [title#17, author#18, call_number#19, item_id#20, item_type_code#21, age#24, checkout_library#25, date#26]
:        +- Relation [title#17,author#18,call_number#19,item_id#20,item_type_code#21,status#22,language#23,age#24,checkout_library#25,date#26] csv
+- Relation [library_code#117,library_name#118] csv

== Analyzed Logical Plan ==
library_code: string, title: string, author: string, call_number: string, item_id: string, item_type_code: string, age: string, date: timestamp, library_name: string
Project [library_code#130, title#17, author#18, call_number#1

In [12]:
print(df.count())
print(df.count())
print(df.count())
print(df.count())

59593
59593
59593
59593


In [13]:
df.cache()
print(df.count())

59593


In [14]:
df.select("test").show()

AnalysisException: Column 'test' does not exist. Did you mean one of the following? [age, date, title, author, item_id, call_number, item_type_code, library_code, library_name];
'Project ['test]
+- Project [library_code#130, title#17, author#18, call_number#19, item_id#20, item_type_code#21, age#24, date#121, library_name#118]
   +- Join LeftOuter, (library_code#130 = library_code#117)
      :- Project [title#17, author#18, call_number#19, item_id#20, item_type_code#21, age#24, checkout_library#25 AS library_code#130, date#121]
      :  +- Project [title#17, author#18, call_number#19, item_id#20, item_type_code#21, age#24, checkout_library#25, to_timestamp(date#26, Some(yyyyMMddHHmmss), TimestampType, Some(Australia/Sydney)) AS date#121]
      :     +- Project [title#17, author#18, call_number#19, item_id#20, item_type_code#21, age#24, checkout_library#25, date#26]
      :        +- Relation [title#17,author#18,call_number#19,item_id#20,item_type_code#21,status#22,language#23,age#24,ch

In [15]:
df.withColumn("test", F.max(F.col("library_name"))).show()

AnalysisException: grouping expressions sequence is empty, and 'library_code' is not an aggregate function. Wrap '(max(library_name) AS test)' in windowing function(s) or wrap 'library_code' in first() (or first_value) if you don't care which value you get.;
Aggregate [library_code#130, title#17, author#18, call_number#19, item_id#20, item_type_code#21, age#24, date#121, library_name#118, max(library_name#118) AS test#1180]
+- Project [library_code#130, title#17, author#18, call_number#19, item_id#20, item_type_code#21, age#24, date#121, library_name#118]
   +- Join LeftOuter, (library_code#130 = library_code#117)
      :- Project [title#17, author#18, call_number#19, item_id#20, item_type_code#21, age#24, checkout_library#25 AS library_code#130, date#121]
      :  +- Project [title#17, author#18, call_number#19, item_id#20, item_type_code#21, age#24, checkout_library#25, to_timestamp(date#26, Some(yyyyMMddHHmmss), TimestampType, Some(Australia/Sydney)) AS date#121]
      :     +- Proj